# Keras- the Happy House


In [1]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from kt_utils import *

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline

Using TensorFlow backend.


In [2]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# Normalizing image vectors
X_train = X_train_orig/255.
X_test = X_test_orig/255.

# Reshaping
Y_train = Y_train_orig.T
Y_test = Y_test_orig.T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 600
number of test examples = 150
X_train shape: (600, 64, 64, 3)
Y_train shape: (600, 1)
X_test shape: (150, 64, 64, 3)
Y_test shape: (150, 1)


In [3]:


def HappyModel(input_shape):
    """
    Implementation of the HappyModel.
    
    Arguments:
    input_shape -- shape of the images of the dataset

    Returns:
    model -- a Model() instance in Keras
    """
    
    X_input = Input(input_shape)

    # Zero-Padding: pads the border of X_input with zeroes
    X = ZeroPadding2D((3, 3))(X_input)

    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0')(X)
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
    X = Activation('relu')(X)

    # MAXPOOL
    X = MaxPooling2D((2, 2), name='max_pool')(X)

    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    X = Flatten()(X)
    X = Dense(1, activation='sigmoid', name='fc')(X)

    
    model = Model(inputs = X_input, outputs = X, name='HappyModel')
    
   
    return model

In [4]:

happyModel = HappyModel([64,64,3])

In [5]:
happyModel.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])


In [6]:
happyModel.fit(x=X_train,y=Y_train,epochs=40,batch_size=16)

Epoch 1/40
600/600 [==============================] - 12s - loss: 0.8565 - acc: 0.7350    
Epoch 2/40
600/600 [==============================] - 12s - loss: 0.2449 - acc: 0.9167    
Epoch 3/40
600/600 [==============================] - 12s - loss: 0.1722 - acc: 0.9350    
Epoch 4/40
600/600 [==============================] - 12s - loss: 0.1627 - acc: 0.9350    
Epoch 5/40
600/600 [==============================] - 12s - loss: 0.1152 - acc: 0.9600    
Epoch 6/40
600/600 [==============================] - 12s - loss: 0.1031 - acc: 0.9717    
Epoch 7/40
600/600 [==============================] - 12s - loss: 0.1595 - acc: 0.9350    
Epoch 8/40
600/600 [==============================] - 12s - loss: 0.0754 - acc: 0.9750    
Epoch 9/40
600/600 [==============================] - 12s - loss: 0.0742 - acc: 0.9800    
Epoch 10/40
600/600 [==============================] - 12s - loss: 0.0444 - acc: 0.9883    
Epoch 11/40
600/600 [==============================] - 12s - loss: 0.0608 - acc: 0.9767  

In [7]:
preds = happyModel.evaluate(X_test,Y_test)
print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

150/150 [==============================] - 1s     

Loss = 0.249986527761
Test Accuracy = 0.92666667064


it should be observed much better than random-guessing (50%) accuracy on the train and test sets.

To give  a point of comparison, our model gets around **95% test accuracy in 40 epochs** (and 99% train accuracy) with a mini batch size of 16 and "adam" optimizer. 


In [ ]:
img_path = 'images/my_image.jpg'

img = image.load_img(img_path, target_size=(64, 64))
imshow(img)

x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

print(happyModel.predict(x))

In [ ]:
happyModel.summary()

In [ ]:
plot_model(happyModel, to_file='HappyModel.png')
SVG(model_to_dot(happyModel).create(prog='dot', format='svg'))